In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import DatasetFolder
import numpy as np

class ToTensor(object):
    def __call__(self, image):
        image = image[np.newaxis]
        image = torch.from_numpy(image / 255.0)
        return image

def slice_loader(path):
    return np.load(path)
# torchvision.datasets.DatasetFolder
data = DatasetFolder(root='data3d',loader=slice_loader, transform=ToTensor(),extensions=['.npy'])
#data = ImageFolder(root='data',transform = transforms.ToTensor())

In [2]:
data

Dataset DatasetFolder
    Number of datapoints: 15625
    Root Location: data3d
    Transforms (if any): <__main__.ToTensor object at 0x7ff240329a20>
    Target Transforms (if any): None

In [3]:
data.classes

['empty', 'protein']

In [4]:
# determining if there are any 3d volumes of the wrong shape (there used to be at the edges until I modified
# the slice extraction)
#for i in range (len(data)):
 #   if data[i][0].shape != torch.Size([48, 48, 48]):
 #       print(i, data[i][0].shape)

In [5]:
######################################
# splitting data into train and test sets, randomly, in the ratio 4:1 respectively

In [6]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [7]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)
classes = ('empty','protein')

In [8]:
# displaying random pictures from the training dataset 
#import matplotlib.pyplot as plt
#import numpy as np

# functions to show an image


#def imshow(img):
#    img = img / 2 + 0.5     # unnormalize
#    npimg = img.numpy()
#    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
print(images[0].shape)

empty protein empty empty
torch.Size([1, 48, 48, 48])


In [9]:
# creating the network
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1, 6, (5,5,5))
        self.pool = nn.MaxPool3d(2, 2)
        self.conv2 = nn.Conv3d(6, 16, (5,5,5))
        self.fc1 = nn.Linear(9*9*9*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x = x.view(-1, 9*9*9*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.501
[2,  2000] loss: 0.084
Finished Training


In [ ]:
#dataiter = iter(testloader)
#images, labels = dataiter.next()

# print images
#imshow(torchvision.utils.make_grid(images))
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [12]:
outputs = net(images)
outputs

tensor([[ 2.0084, -2.1924],
        [-7.7042,  7.6316],
        [ 1.9475, -2.1265],
        [ 2.0398, -2.2260]], grad_fn=<ThAddmmBackward>)

In [13]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:  empty protein empty empty


In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 97 %
